# Schelling 隔离模型 - 扩展功能演示

本 notebook 演示如何使用扩展的 Schelling 模型，包括：
1. 多种邻域配置（4邻域、8邻域、24邻域）
2. 可自定义的效用函数
3. 两种实现方式的对比（函数式 vs 面向对象）
4. 性能对比

In [ ]:
%cd lectures/lec04/schelling/

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
1
from model import Schelling
from neighborhoods import NeighborhoodType, visualize_neighborhood
from utility_classes import (
    ThresholdUtility,
    LinearUtility,
    QuadraticUtility,
    PeakedUtility,
    SigmoidUtility,
    visualize_utility,
    compare_utilities,
)

## 1. 邻域类型演示

首先了解不同的邻域类型

In [ ]:
# 4邻域 (Von Neumann)
visualize_neighborhood(NeighborhoodType.VON_NEUMANN, radius=1)

In [ ]:
# 8邻域 (Moore)
visualize_neighborhood(NeighborhoodType.MOORE, radius=1)

In [ ]:
# 24邻域 (Extended)
visualize_neighborhood(NeighborhoodType.EXTENDED, radius=2)

## 2. 效用函数演示

理解不同效用函数的特性

In [ ]:
# 创建不同的效用函数
utilities = [
    ThresholdUtility(threshold=0.375),
    LinearUtility(),
    QuadraticUtility(power=2),
    SigmoidUtility(threshold=0.5, steepness=10),
    PeakedUtility(optimal_fraction=0.5, tolerance=0.2),
]

# 比较它们
compare_utilities(utilities, total_neighbors=8)

## 3. 基本模型运行

### 3.1 使用默认配置（8邻域 + 阈值效用函数）

In [ ]:
# 创建模型
model1 = Schelling(
    height=20,
    width=20,
    density=0.8,
    minority_pc=0.5,
    homophily=0.3,  # 阈值
    neighborhood_type=NeighborhoodType.MOORE,
    seed=42,
)

# 运行模型
for i in range(100):
    model1.step()
    if not model1.running:
        break

print(f"模型在第 {model1.steps} 步停止")
print(f"快乐的智能体数量: {model1.happy}/{len(model1.agents)}")

In [ ]:
# 可视化结果
model_data = model1.datacollector.get_model_vars_dataframe()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 快乐智能体数量
ax1.plot(model_data["happy"], label="Happy Agents")
ax1.set_xlabel("Step")
ax1.set_ylabel("Number of Happy Agents")
ax1.set_title("Happy Agents Over Time")
ax1.legend()
ax1.grid(True)

# 快乐智能体百分比
ax2.plot(model_data["pct_happy"], label="% Happy", color="green")
ax2.set_xlabel("Step")
ax2.set_ylabel("Percentage (%)")
ax2.set_title("Percentage of Happy Agents")
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

### 3.2 使用4邻域

In [ ]:
model2 = Schelling(
    height=20,
    width=20,
    density=0.8,
    minority_pc=0.5,
    homophily=0.3,
    neighborhood_type=NeighborhoodType.VON_NEUMANN,  # 4邻域
    seed=42,
)

for i in range(100):
    model2.step()
    if not model2.running:
        break

print(f"4邻域模型在第 {model2.steps} 步停止")
print(f"快乐的智能体数量: {model2.happy}/{len(model2.agents)}")

### 3.3 使用24邻域

In [ ]:
model3 = Schelling(
    height=20,
    width=20,
    density=0.8,
    minority_pc=0.5,
    homophily=0.3,
    neighborhood_type=NeighborhoodType.EXTENDED,  # 24邻域
    radius=2,
    seed=42,
)

for i in range(100):
    model3.step()
    if not model3.running:
        break

print(f"24邻域模型在第 {model3.steps} 步停止")
print(f"快乐的智能体数量: {model3.happy}/{len(model3.agents)}")

## 4. 使用不同的效用函数

### 4.1 线性效用函数

In [ ]:
model4 = Schelling(
    height=20,
    width=20,
    density=0.8,
    minority_pc=0.5,
    utility_function=LinearUtility(),  # 线性效用函数
    seed=42,
)

for i in range(100):
    model4.step()
    if not model4.running:
        break

print(f"线性效用模型在第 {model4.steps} 步停止")
print(f"快乐的智能体数量: {model4.happy}/{len(model4.agents)}")

### 4.2 峰值效用函数（偏好多样性）

In [ ]:
model5 = Schelling(
    height=20,
    width=20,
    density=0.8,
    minority_pc=0.5,
    utility_function=PeakedUtility(
        optimal_fraction=0.5, tolerance=0.15
    ),  # 偏好50%的相似度
    seed=42,
)

for i in range(100):
    model5.step()
    if not model5.running:
        break

print(f"峰值效用模型在第 {model5.steps} 步停止")
print(f"快乐的智能体数量: {model5.happy}/{len(model5.agents)}")

## 5. 参数扫描实验

### 5.1 不同邻域类型的对比

In [ ]:
from mesa.batchrunner import batch_run

# 定义参数
params = {
    "height": 20,
    "width": 20,
    "density": 0.8,
    "minority_pc": 0.5,
    "homophily": 0.3,
    "neighborhood_type": [
        NeighborhoodType.VON_NEUMANN,
        NeighborhoodType.MOORE,
        NeighborhoodType.EXTENDED,
    ],
}

# 运行批处理
results = batch_run(
    Schelling,
    parameters=params,
    iterations=5,
    max_steps=100,
)

df_neighborhood = pd.DataFrame(results)
df_neighborhood.head()

In [ ]:
# 可视化不同邻域类型的影响
neighborhood_names = {
    NeighborhoodType.VON_NEUMANN: "4-Neighborhood",
    NeighborhoodType.MOORE: "8-Neighborhood",
    NeighborhoodType.EXTENDED: "24-Neighborhood",
}

df_neighborhood["neighborhood_name"] = df_neighborhood["neighborhood_type"].map(
    neighborhood_names
)

# 计算最终状态的统计
final_stats = (
    df_neighborhood.groupby(["neighborhood_name", "iteration"]).last().reset_index()
)

plt.figure(figsize=(10, 6))
for name in neighborhood_names.values():
    data = final_stats[final_stats["neighborhood_name"] == name]
    plt.scatter([name] * len(data), data["pct_happy"], alpha=0.6, s=100, label=name)

plt.xlabel("Neighborhood Type")
plt.ylabel("% Happy Agents (Final)")
plt.title("Impact of Neighborhood Type on Segregation")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. 自定义效用函数

演示如何创建自己的效用函数

In [ ]:
from utility_classes import BaseUtility


class CustomUtility(BaseUtility):
    """自定义效用函数：只有在相似度恰好为40%-60%时才满意"""

    def __init__(self, min_fraction=0.4, max_fraction=0.6):
        super().__init__()
        self.min_fraction = min_fraction
        self.max_fraction = max_fraction

    def calculate(self, similar_count: int, total_count: int) -> float:
        if total_count == 0:
            return 0.0

        similarity_fraction = similar_count / total_count

        if self.min_fraction <= similarity_fraction <= self.max_fraction:
            return 1.0
        else:
            return 0.0


# 测试自定义效用函数
custom_utility = CustomUtility(min_fraction=0.4, max_fraction=0.6)
visualize_utility(custom_utility, max_neighbors=8)

In [ ]:
# 使用自定义效用函数运行模型
model6 = Schelling(
    height=20,
    width=20,
    density=0.8,
    minority_pc=0.5,
    utility_function=custom_utility,
    seed=42,
)

for i in range(100):
    model6.step()
    if not model6.running:
        break

print(f"自定义效用模型在第 {model6.steps} 步停止")
print(f"快乐的智能体数量: {model6.happy}/{len(model6.agents)}")

## 7. 总结

通过本 notebook，你学习了：

1. **邻域配置**：4邻域（Von Neumann）、8邻域（Moore）、24邻域（Extended）
2. **效用函数**：阈值、线性、二次、峰值、Sigmoid等多种效用函数
3. **模型扩展**：如何使用不同配置运行模型
4. **参数实验**：如何进行参数扫描和对比实验
5. **自定义**：如何创建自己的效用函数

### 练习建议

1. 尝试不同的效用函数参数组合
2. 比较不同邻域类型对隔离程度的影响
3. 创建你自己的效用函数
4. 研究不同初始密度和少数派比例的影响